## Aula 5.2.Coleta de dados fundamentalistas

### Exemplo 1 - Coletando a lista de companhias com registro na CVM

Acessar o site https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx

In [ ]:
import requests_html

In [ ]:
r = requests_html.HTMLSession().get("https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx")

In [ ]:
r.content

In [ ]:
import re

In [ ]:
a = re.findall(r'(?<=<input name="hdnEmpresas" type="hidden" id="hdnEmpresas" value=")[^"]*(?=" />)', r.text)


In [ ]:
a = a[0]

In [ ]:
list(eval(a.replace("{ key:", "{'key':").replace("value:", "'value':")))

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(list(eval(a.replace("{ key:", "{'key':").replace("value:", "'value':"))))

In [ ]:
df_empresas = pd.DataFrame(list(eval(a.replace("{ key:", "{'key':").replace("value:", "'value':"))))

### Exemplo 2 - Coleta de dados da CVM simulando navegador (selenium)

In [ ]:
url = "https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx"

In [ ]:
!pip install selenium

In [ ]:
!pip install webdriver_manager

In [ ]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
driver.get(url)

In [ ]:
for int_, row in df_empresas.iterrows():
    if "mobly" in row["value"].lower():
        print(row["value"])
        str_mbly = row["value"]

In [ ]:
from selenium.webdriver.common.by import By

In [ ]:
driver.find_element(By.XPATH, "//input[@id='cboEmpresa']").send_keys(str_mbly)

In [ ]:
driver.find_element(By.XPATH, "//li[@tabindex='-1']").click()

In [ ]:
from selenium.webdriver.support.ui import Select

In [ ]:
dropdown = Select(driver.find_element(By.XPATH, "//select[@name='cboCategorias']"))

In [ ]:
driver.find_element(By.XPATH, "//a[@class='chosen-single']").click()

In [ ]:
driver.find_element(By.XPATH, "//input[@id='cboCategorias_chosen_input']").send_keys("ITR")

In [ ]:
driver.find_element(By.XPATH, "//li[@data-option-array-index='41']").click()

In [ ]:
driver.find_element(By.XPATH, "//input[@id='rdPeriodo']").click()

In [ ]:
driver.find_element(By.XPATH, "//input[@id='txtDataIni']").send_keys("01/03/2022")

In [ ]:
driver.find_element(By.XPATH, "//input[@id='txtDataFim']").send_keys("01/03/2023")

In [ ]:
driver.find_element(By.XPATH, "//input[@value='Consultar']").click()

In [ ]:
driver.find_elements(By.XPATH, "//table[@id='grdDocumentos']")

In [ ]:
table_element = driver.find_elements(By.XPATH, "//table[@id='grdDocumentos']")[0]

In [ ]:
for index, row in enumerate(table_element.find_elements(By.CSS_SELECTOR, 'tr')):
    if index > 0:
        if "Assunto" not in row.text:
            a = row.find_elements(By.CSS_SELECTOR, 'td')
            cod_cvm = a[0].text
            empresa = a[1].text
            categoria = a[2].text
            dt_ref = a[5].text
            dt_entrega = a[6].text

In [ ]:
empresa, dt_ref

In [ ]:
a[-1].find_elements(By.XPATH, "//i[@class='fi-download']")

In [ ]:
a[-1].find_elements(By.XPATH, "//i[@class='fi-download']")[0].click()

In [ ]:
b = a[-1].find_elements(By.XPATH, "//i[@class='fi-download']")[0]

In [ ]:
b.get_attribute("onclick")

### Exemplo 2 - Coleta de dados da CVM usando requests-html

No exemplo a seguir vamos primeiramente entender como o site de documentos da CVM funciona.

Vamos inicialmente inspecionar o site https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx

In [ ]:
url = "https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx/ListarDocumentos"

In [ ]:
payload = { 
    "dataDe": '01/03/2022', 
    "dataAte": '01/03/2023' , 
    "empresa": ',025461', 
    "setorAtividade": '-1', 
    "categoriaEmissor": '-1', 
    "situacaoEmissor": '-1', 
    "tipoParticipante": '-1', 
    "dataReferencia": '', 
    "categoria": 'EST_3', 
    "periodo": '2', 
    "horaIni": '', 
    "horaFim": '', 
    "palavraChave":'',
    "ultimaDtRef":'false', 
    "tipoEmpresa":'0', 
    "token": '', 
    "versaoCaptcha": ''
}

In [ ]:
import requests_html

In [ ]:
r = requests_html.HTMLSession().post(url, json=payload)

In [ ]:
r.json()

In [ ]:
dados = r.json()["d"]["dados"]

In [ ]:
dados

In [ ]:
import re
import pandas as pd

In [ ]:
regex = r"(?<=02546-1)(.*?)(?=\$&&\*)"
matches = re.findall(regex, dados)

In [ ]:
len(matches)

In [ ]:
df_docs = pd.DataFrame()

for match in matches:
    
    # Extrair as datas
    
    re_datas = re.findall(r'<spanOrder>(.*?)</spanOrder>', match)
    dt_referencia = re_datas[1]
    dt_entrega = re_datas[2]
    
    # Extrair o nome da cia
    re_cia = re.findall(r"\$&(.+?)\.\$&", match)
    nome_cia = re_cia[0]
    
    # Extrair o vetor com parametros do documento
    re_doc = re.findall(r'</i>(.*?)</i>', match)
    downl_args = re.findall(r"OpenDownloadDocumentos\('(.+?)'\)", re_doc[0])[0].replace("'", "").split(",")
    
    df_docs = df_docs.append(
        pd.DataFrame.from_dict(
            [
                {
                    "Empresa": nome_cia, 
                    "DT_REF": dt_referencia, 
                    "DT_ENTREGA": dt_entrega, 
                    "ARGS": downl_args
                }
            ]
        ), 
        ignore_index=True
    )

In [ ]:
df_docs

In [ ]:
args = df_docs["ARGS"].iloc[0]
args

In [ ]:
url = "https://www.rad.cvm.gov.br/ENET/"

numSequencia = args[0]
numVersao = args[1]
numProtocolo = args[2]
descTipo = args[3]

url = url + "frmDownloadDocumento.aspx?Tela=ext&numSequencia=" + numSequencia + "&numVersao=" + numVersao + "&numProtocolo=" + numProtocolo + "&descTipo=" + descTipo + "&CodigoInstituicao=1";

In [ ]:
url

In [ ]:
requests_html.HTMLSession().get(url)

In [ ]:
numProtocolo

In [ ]:
r = requests_html.HTMLSession().get(url)
with open(f"{numProtocolo}.zip", 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)

In [ ]:
import zipfile

archive = zipfile.ZipFile(f"{numProtocolo}.zip", 'r')
xml_itr = archive.read('025461ITR31-03-2022v1.xml')

In [ ]:
xml_itr

In [ ]:
!pip install xmltodict

In [ ]:
import xmltodict

In [ ]:
my_dict = xmltodict.parse(xml_itr)

In [ ]:
my_dict

In [ ]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario'].keys()

In [ ]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas']

In [ ]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'].keys()

#### Balanço Patrimonial Ativo

In [ ]:
doc = "BalancoPatrimonialAtivo"

In [ ]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]

In [ ]:
df_bpa = pd.DataFrame()
for row in my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]['Conta']:
    df_bpa = pd.concat([df_bpa, pd.DataFrame([row])])

In [ ]:
df_bpa

#### Balanço Patrimonial Passivo

In [ ]:
doc = "BalancoPatrimonialPassivo"

In [ ]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]

In [ ]:
df_bpp = pd.DataFrame()
for row in my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]['Conta']:
    df_bpp = pd.concat([df_bpp, pd.DataFrame([row])])

In [ ]:
df_bpp